In [17]:
!pip install requests
!pip install tweepy
!pip install praw

In [26]:
# TWITTER API CONF VALUES
CONSUMER_KEY=""
CONSUMER_SECRET=""
ACCESS_TOKEN=""
ACCESS_TOKEN_SECRET=""

# PRAW API CONF VALUES
REDDIT_API_INSTANCE=""

# Reddit query
SUBREDDIT_NAME="pics"
LIMIT_POSTS=2
ORDER_BY="top"
TOP_TIME="day"

ADDITIONAL_TEXT="Hello Github! :)" # If you want to include additional text after the Reddit Post title to the tweet 

In [27]:
import requests
import mimetypes
import tweepy
import time
import praw
import os
import glob
import os.path


def getTop(reddit, topTday, LIMIT_POSTS, SUBREDDIT_NAME, ORDER_BY, TOP_TIME):
    invalid_formats = []
    #     for submission in reddit.subreddit("formuladank").new(limit=LIMIT_POSTS):
    #     for submission in reddit.subreddit("formuladank").top(TOP_TIME, limit=LIMIT_POSTS):
    #     for submission in reddit.subreddit("formuladank").rising(limit=LIMIT_POSTS):
    if ORDER_BY == "top":
        for submission in reddit.subreddit(SUBREDDIT_NAME).top(TOP_TIME, limit=LIMIT_POSTS):
            extension = os.path.splitext(submission.url)[1]
            if str(extension) == ".jpg" or str(extension) == ".png":
                if "i.redd.it" in str(submission.url):
                    topTday[(submission.title)] = submission.url
        return topTday


def emptyFolder():
    files = glob.glob('~/Reddit2Twitter/images/*')
    for f in files:
        os.remove(f)


def authentificate(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET):
    twitter_auth_keys = {
        "consumer_key": CONSUMER_KEY,
        "consumer_secret": CONSUMER_SECRET,
        "access_token": ACCESS_TOKEN,
        "access_token_secret": ACCESS_TOKEN_SECRET
    }

    auth = tweepy.OAuthHandler(
        twitter_auth_keys['consumer_key'],
        twitter_auth_keys['consumer_secret']
    )

    auth.set_access_token(
        twitter_auth_keys['access_token'],
        twitter_auth_keys['access_token_secret']
    )

    api = tweepy.API(auth, wait_on_rate_limit=True,
                     wait_on_rate_limit_notify=True)
    return api


def postEvery30seconds(api, topTday):
    for x, y in topTday.items():
        print(x, y)
        # name = input("Enter your name: ")
        # if name == "y":
        img_data = requests.get(y)
        extension = os.path.splitext(y)[1]
        result = ''.join(e for e in x if e.isalnum())
        with open('images/' + result[:5].strip() + str(extension), 'wb') as handler:
            handler.write(img_data.content)
        media = api.media_upload(
            'images/' + result[:5].strip() + str(extension))
        tweet = x
        post_result = api.update_status(
            status=tweet + "\n" + ADDITIONAL_TEXT, media_ids=[media.media_id])
        time.sleep(30) # change this to whatever time you want to wait until new post
    emptyFolder()


def follow(api):
    i = 0
    for follower in api.followers(""):
        time.sleep(60)
        print(follower.screen_name)
        try:
            api.create_friendship(follower.id)
            i += 1
            if i > 240:
                return i
        except TweepError:
            continue


if __name__ == "__main__":
    api = authentificate(CONSUMER_KEY,CONSUMER_SECRET,ACCESS_TOKEN,ACCESS_TOKEN_SECRET)
    reddit = praw.Reddit(REDDIT_API_INSTANCE)

    topPosts = getTop(reddit, {}, LIMIT_POSTS, SUBREDDIT_NAME, ORDER_BY, TOP_TIME)
    postEvery30seconds(api, topPosts)


Protests in NYC to end violence towards Asians. https://i.redd.it/z2wtz6u28vi61.jpg
Someone sent a mariachi band to Ted Cruz's house today https://i.redd.it/flr8ljekixi61.jpg
